<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [2]:
!pip install bertopic[visualization]

zsh:1: no matches found: bertopic[visualization]


In [1]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

/Users/cristian/Desktop/GitHub/clustering/env-tm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect your Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [37]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q319_sust_rice'

analysis_id = 'a01_tm__f01_e01__km02'

# Filtered label
settings_directive = "settings_analysis_directive_2024-12-18-21-48.json"

In [38]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [39]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [40]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [41]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [42]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [43]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}")
print(df.shape)
df.head()

(3480, 42)


,X_N,uuid,PT,AU,AF,TI,SO,LA,DT,DE,...,AR,DI,PG,WC,SC,OA,UT,Countries,IsoCountries,Institutions
0,1,541274e8-5e4e-49fd-ac27-cfecb10be780,J,"Xie, EZ; Zhang, YX; Huang, B; Zhao, YC; Shi, X...","Xie, Enze; Zhang, Yanxia; Huang, Biao; Zhao, Y...",Spatiotemporal variations in soil organic carb...,SOIL & TILLAGE RESEARCH,English,Article,Soil organic carbon; Drivers of SOC; Land use ...,...,104763,10.1016/j.still.2020.104763,11,Soil Science,Agriculture,NaN,WOS:000582696400017,peoples r china,CHN,chinese acad sci; univ chinese acad sci; nanji...
1,2,f79bc6f6-59ec-4773-8f1c-595f7ddc0f9b,J,"Ding, MJ; Guan, QH; Li, LH; Zhang, HM; Liu, C;...","Ding, Mingjun; Guan, Qihui; Li, Lanhui; Zhang,...",Phenology-Based Rice Paddy Mapping Using Multi...,REMOTE SENSING,English,Article,rice paddies; Landsat; MODIS; fusion algorithm...,...,1022,10.3390/rs12061022,18,"Environmental Sciences; Geosciences, Multidisc...",Environmental Sciences & Ecology; Geology; Rem...,gold,WOS:000526820600124,peoples r china,CHN,jiangxi normal univ; xiamen univ technol; sun ...
2,3,f6464579-084d-4f17-9052-464ce88bf459,J,"Masuda, K","Masuda, Kiyotaka",Eco-Efficiency Assessment of Intensive Rice Pr...,SUSTAINABILITY,English,Article,eco-efficiency; life cycle assessment; global ...,...,5368,10.3390/su11195368,14,Green & Sustainable Science & Technology; Envi...,Science & Technology - Other Topics; Environme...,"Green Published, gold",WOS:000493525500227,japan,JPN,univ shiga prefecture
3,4,d7436b41-8ad3-4f4a-a934-40af5a47e9d4,J,"Malyan, SK; Bhatia, A; Kumar, SS; Fagodiya, RK...","Malyan, Sandeep K.; Bhatia, Arti; Kumar, Smita...",Mitigation of greenhouse gas intensity by supp...,BIOCATALYSIS AND AGRICULTURAL BIOTECHNOLOGY,English,Article,Rice; Azolla; Methane; Nitrous oxide; Yield-sc...,...,101266,10.1016/j.bcab.2019.101266,8,Biotechnology & Applied Microbiology,Biotechnology & Applied Microbiology,NaN,WOS:000493911600089,india; vietnam; israel,IND; VNM; ISR,icar indian agr res inst; indian inst technol ...
4,5,d1e361d9-aefd-498c-833c-a84705c30488,J,"Srivastava, P; Gupta, S; Garaniya, V; Abbassi,...","Srivastava, Pratiksha; Gupta, Supriya; Garaniy...",Up to 399mV bioelectricity generated by a rice...,ENVIRONMENTAL CHEMISTRY LETTERS,English,Article,Plant microbial fuel cell; Microbial fuel cell...,...,NaN,10.1007/s10311-018-00824-2,7,"Chemistry, Multidisciplinary; Engineering, Env...",Chemistry; Engineering; Environmental Sciences...,NaN,WOS:000468974600029,australia; india,AUS; IND,univ tasmania; macquarie univ; csir inst miner...




---



## PART 2: Topic Model

In [44]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [45]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [46]:
# Combine embeddings
documents = corpus.text.to_list()

In [47]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [48]:
len(documents)

3480

In [49]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [50]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [51]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2024-12-18 21:49:20,233 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-18 21:49:29,559 - BERTopic - Dimensionality - Completed ✓
2024-12-18 21:49:29,560 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-18 21:49:29,866 - BERTopic - Cluster - Completed ✓
2024-12-18 21:49:29,870 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-18 21:49:33,313 - BERTopic - Representation - Completed ✓


In [52]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,0,131,0_the_and_data_of,"[the, and, data, of, to, rice, mapping, in, fo...",[Phenology-Based Rice Paddy Mapping Using Mult...
1,1,116,1_grazing_of_the_and,"[grazing, of, the, and, to, in, pasture, manag...","[Balancing animal, pasture and environmental o..."
2,2,101,2_cd_and_soil_the,"[cd, and, soil, the, of, in, heavy, cadmium, r...",[Vermicompost Supply Enhances Fragrant-Rice Yi...
3,3,101,3_biochar_and_soil_of biochar,"[biochar, and, soil, of biochar, of, the, in, ...",[Comparative Metagenomics Reveals Enhanced Nut...
4,4,99,4_soil_bacterial_microbial_community,"[soil, bacterial, microbial, community, and, t...",[Changes in soil bacterial community and enzym...
5,5,95,5_farmers_the_of_to,"[farmers, the, of, to, and, in, adoption, for,...",[The ironies of new innovation and the sunset ...
6,6,94,6_rice_and_of_the,"[rice, and, of, the, in, to, yield, for, crop,...",[Meeting the challenges of global rice product...
7,7,93,7_emissions_and_the_ch4,"[emissions, and, the, ch4, in, n2o, of, ghg, r...",[Advancing effective methods for mitigating gr...
8,8,87,8_and_of_the_yield,"[and, of, the, yield, rice, soil, in, fertiliz...",[Manure applications combined with chemical fe...
9,9,86,9_soil_organic_and_of,"[soil, organic, and, of, carbon, the, in, soc,...",[Organic fertilizer made from food waste impro...


In [53]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [54]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

59

In [55]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [56]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('the', 0.012358549365124002),
 ('and', 0.01099488008115488),
 ('data', 0.009542186131471127),
 ('of', 0.009364639742845765),
 ('to', 0.008954350309158355),
 ('rice', 0.00888742343302793),
 ('mapping', 0.008558701298137591),
 ('in', 0.00817366173532818),
 ('for', 0.007921013751405758),
 ('paddy', 0.007339595145542714)]

In [57]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['Phenology-Based Rice Paddy Mapping Using Multi-Source Satellite Imagery and a Fusion Algorithm Applied to the Poyang Lake Plain, Southern China Accurate information about the spatiotemporal patterns of rice paddies is essential for the assessment of food security, management of agricultural resources, and sustainability of ecosystems. However, accurate spatial datasets of rice paddy fields and multi-cropping at fine resolution are still lacking. Landsat observation is the primary source of remote sensing data that has continuously mapped regional rice paddy fields at a 30-m spatial resolution since the 1980s. However, Landsat data used for rice paddy studies reveals some challenges, especially data quality issues (e.g., cloud cover). Here, we present an algorithm that integrates time-series Landsat and MODIS (Moderate-resolution Imaging Spectroradiometer) images with a phenology-based approach (ILMP) to map rice paddy planting fields and multi-cropping patterns. First, a fusion of MO

In [58]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [59]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': KMeans(n_clusters=59),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [60]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [61]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 4/4 [00:05<00:00,  1.49s/it]


In [62]:
topic_distr

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.89327726, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01475847, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.49956239, ..., 0.        , 0.        ,
        0.        ]])

In [63]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

,UT,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document,Score,X_E,X_C,level0,cl99,cl-99
0,WOS:000582696400017,Spatiotemporal variations in soil organic carb...,29,29_soil_quality_the_and,"[soil, quality, the, and, soil quality, in, of...",[Soil quality assessment of yellow clayey padd...,soil - quality - the - and - soil quality - in...,False,0.156848,0.156848,30,30,False,False
1,WOS:000526820600124,Phenology-Based Rice Paddy Mapping Using Multi...,0,0_the_and_data_of,"[the, and, data, of, to, rice, mapping, in, fo...",[Phenology-Based Rice Paddy Mapping Using Mult...,the - and - data - of - to - rice - mapping - ...,True,0.893277,0.893277,1,1,False,False
2,WOS:000493525500227,Eco-Efficiency Assessment of Intensive Rice Pr...,18,18_energy_the_and_of,"[energy, the, and, of, production, in, to, ric...",[Evaluation of energy balance and greenhouse g...,energy - the - and - of - production - in - to...,False,0.745255,0.745255,19,19,False,False
3,WOS:000493911600089,Mitigation of greenhouse gas intensity by supp...,7,7_emissions_and_the_ch4,"[emissions, and, the, ch4, in, n2o, of, ghg, r...",[Advancing effective methods for mitigating gr...,emissions - and - the - ch4 - in - n2o - of - ...,False,0.328442,0.328442,8,8,False,False
4,WOS:000468974600029,Up to 399mV bioelectricity generated by a rice...,47,47_of_the_concrete_ash,"[of, the, concrete, ash, and, as, silica, husk...",[Heat absorbing glass from rice husk ash for a...,of - the - concrete - ash - and - as - silica ...,False,0.865050,0.865050,48,48,False,False


In [64]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [65]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2024-12-18 21:49:41,544 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

